In [1]:
# blanket import modules
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
import pprint
from scipy.stats import linregress
import requests
import gmaps
import gmaps.geojson_geometries
from api_key import g_key
from matplotlib.colors import to_hex
from matplotlib.cm import inferno
#api key
gmaps.configure(api_key=g_key)

In [2]:
happy_cs = "Output/happy_comb-data.csv"

In [3]:
happy_df = pd.read_csv(happy_cs)
#trying to use this at the end 
happy_country=happy_df['Country']
happy_df

,RANK,Country,Happiness score,Explained by: Healthy life expectancy,Sunshine hours per Year,Average Income,Alcohol Consumption (L/year)
0,1,Finland,7.821,0.775,1858.00,18742.0,10.65
1,2,Denmark,7.636,0.777,1932.00,20304.0,10.13
2,3,Iceland,7.557,0.803,1326.00,19300.0,9.21
3,4,Switzerland,7.512,0.822,1932.75,25787.0,11.23
4,5,Netherlands,7.415,0.787,1662.00,19690.0,9.67
...,...,...,...,...,...,...,...
93,136,India,3.777,0.471,2517.02,1314.0,5.61
94,137,Zambia,3.760,0.306,2965.47,1045.0,4.46
95,138,Malawi,3.750,0.388,2795.87,733.0,4.08
96,139,Tanzania,3.702,0.425,3032.85,992.0,12.04


In [4]:
#making initial map with lines around country to fill later
countries_geojson = gmaps.geojson_geometries.load_geometry('countries')
fig = gmaps.figure()
happy_layer=gmaps.geojson_layer(countries_geojson)
# fig.add_layer(happy_layer)
# fig


In [5]:
# need to get the min and the max of happiness
# happy_df=happy_df.set_index(["Country"])

In [6]:
happy_df.dtypes

RANK                                       int64
Country                                   object
Happiness score                          float64
Explained by: Healthy life expectancy    float64
Sunshine hours per Year                  float64
Average Income                           float64
Alcohol Consumption (L/year)             float64
dtype: object

In [7]:
happy_col=happy_df["Happiness score"]
h_range=[]

for x in happy_col:
    happ_conv= x / 10
    h_range.append(happ_conv)
    
# happy_range=happ_max-happ_min

happy_df['Happiness Range'] = h_range
happy_df
happy_df["Country"].values

array(['Finland', 'Denmark', 'Iceland', 'Switzerland', 'Netherlands',
       'Sweden', 'Norway', 'Israel', 'Austria', 'Australia', 'Ireland',
       'Germany', 'Canada', 'United States of America', 'United Kingdom',
       'Belgium', 'France', 'Slovenia', 'Romania', 'Spain', 'Uruguay',
       'Italy', 'Malta', 'Lithuania', 'Slovakia', 'Estonia', 'Panama',
       'Brazil', 'Kazakhstan', 'Cyprus', 'Latvia', 'Serbia', 'Chile',
       'Nicaragua', 'Mexico', 'Croatia', 'Poland', 'El Salvador',
       'Hungary', 'Uzbekistan', 'Japan', 'Honduras', 'Portugal',
       'Argentina', 'Greece', 'South Korea', 'Philippines', 'Thailand',
       'Moldova', 'Colombia', 'Bosnia and Herzegovina', 'Mongolia',
       'Bolivia', 'China', 'Paraguay', 'Peru', 'Montenegro', 'Ecuador',
       'Vietnam', 'Russia', 'Armenia', 'Bulgaria', 'Ivory Coast',
       'North Macedonia', 'Albania', 'South Africa', 'Bangladesh',
       'Algeria', 'Ukraine', 'Morocco', 'Mozambique', 'Cameroon',
       'Senegal', 'Georgia', '

In [8]:
# We will need to scale the Happiness values to lie between 0 and 1, this sets up for color ranges below
happ_max1=happy_df["Happiness Range"].max()
happ_min1=happy_df["Happiness Range"].min()
hap_range=happ_max1-happ_min1
inferno2= inferno.reversed()

def calculate_color(h_range):
    
#     Convert the GINI coefficient to a color

    # make gini a number between 0 and 1
    normalized_range = (h_range - happ_min1) / hap_range

    # invert gini so that high inequality gives dark color
    inverse_hap = 1.0 - normalized_range

    # transform the gini coefficient to a matplotlib color
    mpl_color = inferno2(inverse_hap)

    # transform from a matplotlib color to a valid CSS color
    gmaps_color = to_hex(mpl_color, keep_alpha=False)

    return gmaps_color


In [9]:
colors = []
for feature in countries_geojson['features']:
    country_name = feature['properties']['name']
    print(country_name)
    try:
        h_range = happy_df.loc[happy_df["Country"]==country_name]["Happiness Range"].values
        print(h_range)
        color = calculate_color(h_range)
        print(color)
    except KeyError:
        # no happyy score for that country: return default color
        color = (0, 0, 0, 0.3)
        print(country_name+" not available")
    except ValueError: 
        color = (0, 0, 0, 0.3)
        
    print(color)    
    colors.append(color)
colors

Afghanistan
[]
(0, 0, 0, 0.3)
Angola
[]
(0, 0, 0, 0.3)
Albania
[0.5199]
#a92e5e
#a92e5e
Aland
[]
(0, 0, 0, 0.3)
Andorra
[]
(0, 0, 0, 0.3)
United Arab Emirates
[]
(0, 0, 0, 0.3)
Argentina
[0.5967]
#e15635
#e15635
Armenia
[0.5399]
#ba3655
#ba3655
American Samoa
[]
(0, 0, 0, 0.3)
Antarctica
[]
(0, 0, 0, 0.3)
French Southern and Antarctic Lands
[]
(0, 0, 0, 0.3)
Antigua and Barbuda
[]
(0, 0, 0, 0.3)
Australia
[0.7162]
#fac62d
#fac62d
Austria
[0.7163]
#fac62d
#fac62d
Azerbaijan
[]
(0, 0, 0, 0.3)
Burundi
[]
(0, 0, 0, 0.3)
Belgium
[0.6805]
#fca108
#fca108
Benin
[0.4623]
#7a1d6d
#7a1d6d
Burkina Faso
[0.467]
#7d1e6d
#7d1e6d
Bangladesh
[0.5155]
#a62d60
#a62d60
Bulgaria
[0.5371]
#b93556
#b93556
Bahrain
[]
(0, 0, 0, 0.3)
The Bahamas
[]
(0, 0, 0, 0.3)
Bosnia and Herzegovina
[0.5768]
#d54a41
#d54a41
Belarus
[]
(0, 0, 0, 0.3)
Belize
[]
(0, 0, 0, 0.3)
Bolivia
[0.56]
#ca404a
#ca404a
Brazil
[0.6293]
#f06f20
#f06f20
Barbados
[]
(0, 0, 0, 0.3)
Brunei
[]
(0, 0, 0, 0.3)
Bhutan
[]
(0, 0, 0, 0.3)
Botswana
[]


[(0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 '#a92e5e',
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 '#e15635',
 '#ba3655',
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 '#fac62d',
 '#fac62d',
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 '#fca108',
 '#7a1d6d',
 '#7d1e6d',
 '#a62d60',
 '#b93556',
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 '#d54a41',
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 '#ca404a',
 '#f06f20',
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 '#fbb61a',
 '#f2e865',
 '#eb6628',
 '#c83f4b',
 '#ad305d',
 '#9d2964',
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 '#d54a41',
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 '#ee6a24',
 (0, 0, 0, 0.3),
 '#fbb81d',
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 '#f2f482',
 (0, 0, 0, 0.3),
 '#a32c61',
 '#c43c4e',
 '#5d126e',
 (0, 0, 0, 0.3),
 '#f68013',
 '#f2741c',
 '#5a116e',
 '#fcffa4',
 (0, 0, 0, 0.3),
 (0, 0, 0, 0.3),
 '#fa9407',

In [10]:

fig = gmaps.figure()
happy_layer2 = gmaps.geojson_layer(
    countries_geojson,
    fill_color=colors,
    stroke_color=colors,
    fill_opacity=0.8)
fig.add_layer(happy_layer2)
fig

Figure(layout=FigureLayout(height='420px'))